In [1]:
# import header files
import torch
import torch.nn as nn
import torchvision
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
np.random.seed(1234)
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)

In [2]:
# define transforms
train_transforms = torchvision.transforms.Compose([torchvision.transforms.RandomRotation(30),
                                       torchvision.transforms.Resize((224, 224)),
                                       torchvision.transforms.RandomHorizontalFlip(),
                                       torchvision.transforms.ToTensor(),
                                       torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
# get data
train_data = torchvision.datasets.ImageFolder("/content/drive/My Drive/train_images/", transform=train_transforms)
val_data = torchvision.datasets.ImageFolder("/content/drive/My Drive/val_images/", transform=train_transforms)
print(len(train_data))
print(len(val_data))

In [ ]:
# data loader
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=16, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=True, num_workers=16, pin_memory=True)

In [3]:
# define loss for two-class problem
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# define model
class ResNet_50(torch.nn.Module):
    
    
    # block 1
    def block_1(self, in_features):
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_features, 64, kernel_size=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(64, 64, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(64, 256, kernel_size=1),
            torch.nn.BatchNorm2d(256)
        )    
        
    # block 2
    def block_2_init(self, in_features):
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_features, 128, kernel_size=1, stride=2),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(128, 128, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(128, 512, kernel_size=1),
            torch.nn.BatchNorm2d(512)
        )
    
    def block_2(self, in_features):
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_features, 128, kernel_size=1),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(128, 128, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(128, 512, kernel_size=1),
            torch.nn.BatchNorm2d(512)
        )
    
    # block 3
    def block_3_init(self, in_features):
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_features, 256, kernel_size=1, stride=2),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 256, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 1024, kernel_size=1),
            torch.nn.BatchNorm2d(1024)
        )
    
    def block_3(self, in_features):
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_features, 256, kernel_size=1),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 256, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 1024, kernel_size=1),
            torch.nn.BatchNorm2d(1024)
        )
    
    # block 4
    def block_4_init(self, in_features):
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_features, 512, kernel_size=1, stride=2),
            torch.nn.BatchNorm2d(512),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(512, 512, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(512),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(512, 2048, kernel_size=1),
            torch.nn.BatchNorm2d(2048)
        )
    
    def block_4(self, in_features):
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_features, 512, kernel_size=1),
            torch.nn.BatchNorm2d(512),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(512, 512, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(512),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(512, 2048, kernel_size=1),
            torch.nn.BatchNorm2d(2048)
        )
    
    
    # init function
    def __init__(self, num_classes = 2):
        super(ResNet_50, self).__init__()
        
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        # block 1
        self.resnet_block_1_1 = self.block_1(64)
        self.resnet_block_1_2 = self.block_1(256)
        self.resnet_block_1_3 = self.block_1(256)
        
        # block 2
        self.resnet_block_2_1 = self.block_2_init(256)
        self.resnet_block_2_2 = self.block_2(512)
        self.resnet_block_2_3 = self.block_2(512)
        self.resnet_block_2_4 = self.block_2(512)
        
        # block 3
        self.resnet_block_3_1 = self.block_3_init(512)
        self.resnet_block_3_2 = self.block_3(1024)
        self.resnet_block_3_3 = self.block_3(1024)
        self.resnet_block_3_4 = self.block_3(1024)
        self.resnet_block_3_5 = self.block_3(1024)
        self.resnet_block_3_6 = self.block_3(1024)
        
        # block 4
        self.resnet_block_4_1 = self.block_4_init(1024)
        self.resnet_block_4_2 = self.block_4(2048)
        self.resnet_block_4_3 = self.block_4(2048)
        
        self.avgpool = torch.nn.AdaptiveAvgPool2d(7)
        
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(2048 * 7 * 7, num_classes)
        )
        
        self.relu = torch.nn.Sequential(
            torch.nn.ReLU(inplace=True)
        )
        
        self.skip_connection_1 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 256, kernel_size=1),
            torch.nn.BatchNorm2d(256)
        )
        
        self.skip_connection_2 = torch.nn.Sequential(
            torch.nn.Conv2d(256, 512, kernel_size=1, stride=2),
            torch.nn.BatchNorm2d(512)
        )
        
        self.skip_connection_3 = torch.nn.Sequential(
            torch.nn.Conv2d(512, 1024, kernel_size=1, stride=2),
            torch.nn.BatchNorm2d(1024)
        )
        
        self.skip_connection_4 = torch.nn.Sequential(
            torch.nn.Conv2d(1024, 2048, kernel_size=1, stride=2),
            torch.nn.BatchNorm2d(2048)
        )
        
        
    # define forward function
    def forward(self, x):
        
        # apply initial conv layers
        x = self.features(x)
        
        # block 1
        x_1 = self.resnet_block_1_1(x)
        x = self.skip_connection_1(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        x_1 = self.resnet_block_1_2(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        x_1 = self.resnet_block_1_3(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        
        # block 2
        x_1 = self.resnet_block_2_1(x)
        x = self.skip_connection_2(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        x_1 = self.resnet_block_2_2(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        x_1 = self.resnet_block_2_3(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        x_1 = self.resnet_block_2_4(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        
        # block 3
        x_1 = self.resnet_block_3_1(x)
        x = self.skip_connection_3(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        x_1 = self.resnet_block_3_2(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        x_1 = self.resnet_block_3_3(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        x_1 = self.resnet_block_3_4(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        x_1 = self.resnet_block_3_5(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        x_1 = self.resnet_block_3_6(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        
        # block 4
        x_1 = self.resnet_block_4_1(x)
        x = self.skip_connection_4(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        x_1 = self.resnet_block_4_2(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        x_1 = self.resnet_block_4_3(x)
        x = torch.add(x, x_1)
        x = self.relu(x)
        
        x = self.avgpool(x)
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)
        return x

In [ ]:
# define model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet_50()
model.to(device)

In [ ]:
# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.001)

In [ ]:
train_losses = []
train_acc = []
val_losses = []
val_acc = []
best_metric = -1
best_metric_epoch = -1

# train and validate
for epoch in range(0, 100):
    
    # train
    model.train()
    training_loss = 0.0
    total = 0
    correct = 0
    for i, (input, target) in enumerate(train_loader):
        
        input = input.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()
        
        training_loss = training_loss + loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()
        
    training_loss = training_loss / float(len(train_loader))
    training_accuracy = str(100.0 * (float(correct) / float(total)))
    train_losses.append(training_loss)
    train_acc.append(training_accuracy)
    
    # validate
    model.eval()
    valid_loss = 0.0
    total = 0
    correct = 0
    for i, (input, target) in enumerate(val_loader):
        
        with torch.no_grad():
            input = input.to(device)
            target = target.to(device)

            output = model(input)
            loss = criterion(output, target)
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            
        valid_loss = valid_loss + loss.item()
    valid_loss = valid_loss / float(len(val_loader))
    valid_accuracy = str(100.0 * (float(correct) / float(total)))
    val_losses.append(valid_loss)
    val_acc.append(valid_accuracy)


    # store best model
    if(float(valid_accuracy) > best_metric and epoch >= 30):
      best_metric = float(valid_accuracy)
      best_metric_epoch = epoch
      torch.save(model.state_dict(), "best_model.pth")
    
    print()
    print("Epoch" + str(epoch) + ":")
    print("Training Accuracy: " + str(training_accuracy) + "    Validation Accuracy: " + str(valid_accuracy))
    print("Training Loss: " + str(training_loss) + "    Validation Loss: " + str(valid_loss))
    print()